## 1.1. Get the list of places
We start with the list of places to include in your corpus of documents. In particular, we focus on the Most popular places. Next, we want you to collect the URL associated with each site in the list from this list. The list is long and split into many pages. Therefore, we ask you to retrieve only the URLs of the places listed in the first 400 pages (each page has 18 places, so that you will end up with 7200 unique place URLs).

The output of this step is a .txt file whose single line corresponds to the place's URL.

In [1]:
import requests
import bs4
import os

Since the first page have a different url we have saved the url in the variable urlP.\
Then the other 399 have a common url, the only different is the number of the page, so we have splitted the url in two parts,\
so every time we want to access a different url we can simply concatenate these two with the number of the page in the middle.

In [29]:
urlP = "https://www.atlasobscura.com/places?sort=likes_count"
url1="https://www.atlasobscura.com/places?page="
url2="&sort=likes_count"

In [51]:
def extract_single_link(url1,url2):
     with open("Places/Address.txt","w") as f:
        for i in range(1,401):
            if i==1:
                result = requests.get(urlP)
                soup = bs4.BeautifulSoup(result.text,"lxml")
                for item in soup.find_all('a',{'class':'content-card content-card-place'}):
                    f.write("https://www.atlasobscura.com"+item["href"]+"\n")
            else:
                i=str(i)
                url = url1+i+url2
                result = requests.get(url)
                soup = bs4.BeautifulSoup(result.text,"lxml")
                for item in soup.find_all('a',{'class':'content-card content-card-place'}):
                    f.write("https://www.atlasobscura.com"+item["href"]+"\n")



In [52]:
extract_single_link(url1,url2)

## 1.2. Crawl places
Once you get all the URLs in the first 400 pages of the list, you:

Download the HTML corresponding to each of the collected URLs.
After you collect a single page, immediately save its HTML in a file. In this way, if your program stops for any reason, you will not lose the data collected up to the stopping point.
Organize the entire set of downloaded HTML pages into folders. Each folder will contain the HTML of the places on page 1, page 2, ... of the list of locations.
Tip: Due to a large number of pages you should download, you can use some methods that can help you shorten the time it takes. If you employed a particular process or approach, kindly describe it.

In [ ]:
with open('Places/Address.txt', 'r') as f:
    urls_places = f.readlines()

for index, url in enumerate(urls_places):                                                   #variable index is useful to take in account the ranking of the place
    page_number = index//18                                                                 #with this variable we are able to allocate the html file in the write folder
    r = requests.get(url[:-1])                                                              
    filename=f'Page_{page_number+1}/{index+1}_{url[36:-1].replace("-"," ")}.html'           #url[36:1] since from position 36 starts the name of the place for every url
    os.makedirs(os.path.dirname(filename), exist_ok=True)                                   #we have to say that this direction exists otherwise we will be have an error
    with open(filename, 'w',encoding="utf-8") as file:
        file.write(r.text)
        file.close